# Proyecto individual ML_Ops Movies

In [1]:
import pandas as pd
from pandas import json_normalize
from datetime import datetime
import ast  # Importar el módulo ast

ETL con el dataset de movies

In [3]:
df_movies = pd.read_csv("c:\\Users\\Giuliano\\Desktop\\movies_dataset.csv",low_memory=False)

Desanidamos belongs_to_collection

In [81]:
# Convertir las cadenas en diccionarios
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else None)


In [82]:
df_desanidado = pd.json_normalize(df_movies['belongs_to_collection'])

In [83]:
# Unir el DataFrame desanidado con el DataFrame original
df_movies_collection= pd.concat([df_movies, df_desanidado], axis=1)

Desanidamos product_companies

In [84]:
# Convertir las cadenas en listas de diccionarios, manejando NaN y None
df_movies['production_companies'] = df_movies['production_companies'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None
)

# Expandir las listas en filas separadas
df_expanded = df_movies.explode('production_companies')
# Convertir los diccionarios en columnas separadas
df_companies = pd.json_normalize(df_expanded['production_companies'])

In [118]:
df_movies = pd.concat([df_movies_collection,df_companies], axis=1)

In [119]:
# Borramos las columnas bellong_to_collection y product_companies
df_movies = df_movies.drop(columns=['belongs_to_collection', 'production_companies'])

In [120]:
# Reemplazar los valores nulos en las columnas 'revenue' y 'budget' por 0
df_movies['revenue'] = df_movies['revenue'].fillna(0)
df_movies['budget'] = df_movies['budget'].fillna(0)

In [88]:
# Eliminar las filas donde el valor de 'release_date' es nulo
df_movies = df_movies.dropna(subset=['release_date'])

In [105]:
# Verificar cuántos valores nulos quedan en 'release_date'
nulos_release_date = df_movies['release_date'].isnull().sum()
nulos_release_date

np.int64(0)

In [124]:
# Asegurarse de que 'release_date' esté en formato de fecha AAAA-mm-dd
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d',errors="coerce")

In [107]:
# Eliminar las filas donde 'release_date' no pudo convertirse (valores nulos)
df_movies = df_movies.dropna(subset=['release_date'])

In [108]:
# Crear una nueva columna 'release_year' extrayendo el año de 'release_date'
df_movies['release_year'] = df_movies['release_date'].dt.year

In [109]:
# Asegurarse de que 'revenue' y 'budget' sean numéricos, reemplazar valores no convertibles por 0
df_movies['revenue'] = pd.to_numeric(df_movies['revenue'], errors='coerce').fillna(0)
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce').fillna(0)

In [94]:
df_movies["retorno_de_inversion"] = df_movies["revenue"] / df_movies["budget"] 

In [95]:
df_movies["retorno_de_inversion"]

0        12.451801
1         4.043035
2              NaN
3         5.090760
4              inf
           ...    
45460          NaN
45462          NaN
45463          NaN
45464          NaN
45465          NaN
Name: retorno_de_inversion, Length: 45376, dtype: float64

In [96]:
df_movies["retorno_de_inversion"] = df_movies["retorno_de_inversion"].fillna(0)

In [98]:
# Si está correcto, elimina las columnas
df_movies = df_movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

In [131]:
# Verificar el resultado
print(df_movies['release_date'].head())

0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10
Name: release_date, dtype: datetime64[ns]


In [101]:
df_movies.to_csv("df_movies.csv")

ETL con el dataset de credits

In [ ]:
df_credits = pd.read_csv("c:\\Users\\Giuliano\\Desktop\\credits.csv")

In [5]:
df_credits["cast"][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [6]:
# Convertir las cadenas en listas de diccionarios, manejando NaN y None
df_credits['cast'] = df_credits['cast'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None
)

In [ ]:
# Convertir las cadenas en listas de diccionarios, manejando NaN y None
df_credits['cast'] = df_credits['cast'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None
)

# Expandir las listas en filas separadas
df_cast = df_credits.explode('production_companies')
# Convertir los diccionarios en columnas separadas
df_cast = pd.json_normalize(df_cast['production_companies'])